In [ ]:
!pip install yfinance
!pip install finta
from finta import TA
import pandas as pd
import yfinance as yf
import glob
import numpy as np

In [ ]:
# Function with a 'folder' and 'years' argument. Purpose is to download and save historic price data for each ticker as .csv and also in a dictionary.
def download_stock_price_data_and_save_to_a_dict(folder='', years=5):

  # 'tickers' is a pandas DataFrame object. 'pd.read_csv' function reads csv file from this website which contains all of the stock ticker symbols under it's "ACT Symbol" column.
  tickers = pd.read_csv('https://pkgstore.datahub.io/core/nyse-other-listings/nyse-listed_csv/data/3c88fab8ec158c3cd55145243fe5fcdf/nyse-listed_csv.csv')
  # 'symbols' is a list of symbols we made by taking all the unique symbols from the tickers .csv
  symbols = tickers['ACT Symbol'].unique()
  # empty dictionary to store all stock price DataFrames.
  price_data_dict = {}

  # loop where we go through each symbol 's' in the symbols list (first 20), and then do some things.
  for s in symbols[0:20]:
    try:

      # creates the DataFrame 'price_data_df' from the downloaded yf price data over the last years. auto_adjust accounts for stock splits and such. Takes in each stock symbol 's' one at at time.
      price_data_df = yf.download([s], auto_adjust=True, period=f'{str(years)}y')

      # If the length of the dataframe is over 100, then go ahead and do the iteration.
      if len(price_data_df) > 100:
        # make the columns lowercase, rename the index to 'datetime', and adds the 'price_data_df' to the 'price_data_dict' with the key being the current symbol 's'
        price_data_df.columns = price_data_df.columns.str.lower()
        price_data_df.index = price_data_df.index.rename('datetime')
        price_data_df.to_csv(f'{folder}{s}.csv')
        price_data_dict[s] = price_data_df

    # error catch
    except Exception as e:
      print(f'error with {s}')
      print(repr(e))
      continue
  
  # Allows for the 'price_data_dict' dictionary object to be accessed elsewhere in the code as a variable or input in another function.   
  return price_data_dict
  
# Download and saves price data to our price data dictionary.
price_data_dict = download_stock_price_data_and_save_to_a_dict()

In [4]:
# takes in 'price_data_df' DataFrames as an argument and transforms it to a new DataFrame later called 'entries'.
def create_entry_df(price_data_df:pd.DataFrame) -> pd.DataFrame:
  # Adding various columns to the original 'df'.
  price_data_df['EMA200'] = TA.EMA(price_data_df, period=200)
  price_data_df['EMA50'] = TA.EMA(price_data_df, period=50)
  price_data_df['ATR'] = TA.ATR(price_data_df)
  # entry is lazily done, it's just the opening price of the next row
  price_data_df['entry'] = price_data_df.shift(-1).open
  price_data_df['target'] = 0.0
  price_data_df['stop'] = 0.0
  price_data_df['risk:reward'] = 0.0
  price_data_df['target_pct'] = 0.0
  price_data_df['stop_pct'] = 0.0
  price_data_df['target_profit'] = 150
  price_data_df['position_size'] = 0.0

  # Conditions for long entry based on technical analysis indicators.
  c1 = price_data_df.close > price_data_df.EMA200
  c2 = (price_data_df.low < price_data_df.EMA50) & (price_data_df.close > price_data_df.EMA50) & (price_data_df.open > price_data_df.EMA50)
  long_entry_condition = (c1) & (c2)

  # Accesses and modifies all rows in 'df' where long_entry_condition is true, and modifies the 2nd input '' column in those rows to what is specified.
  price_data_df.loc[long_entry_condition, 'target'] = price_data_df.entry + (1.5*price_data_df.ATR)
  price_data_df.loc[long_entry_condition, 'stop'] = price_data_df.entry - price_data_df.ATR
  price_data_df.loc[long_entry_condition, 'risk:reward'] = (price_data_df.target - price_data_df.entry) / (price_data_df.entry - price_data_df.stop)
  price_data_df.loc[long_entry_condition, 'target_pct'] = (price_data_df.target - price_data_df.entry) / (price_data_df.entry)
  price_data_df.loc[long_entry_condition, 'stop_pct'] = -(price_data_df.entry - price_data_df.stop) / (price_data_df.entry)
  price_data_df.loc[long_entry_condition, 'position_size'] = price_data_df.target_profit / price_data_df.target_pct

  # Selects the subset of rows where the price data in DataFrame 'price_data_df' satisfy the long entry condition, and then creates a copy of the modified DataFrame denoted 'entries'.
  entries = price_data_df.loc[long_entry_condition][['entry', 'target', 'stop', 'risk:reward', 'target_pct', 'stop_pct', 'target_profit', 'position_size']].copy()

  # Allows for the 'entries' DataFrame to be accessed elsewhere in the code as a variable. 
  return entries

In [5]:
# Create empty list and dictionary for future use.
# Goal of this chunk: Needed to grab all the 'survivor' valid .csv files with actual data, so couldn't just grab every single symbol from the 'symbols' array to plug into the trade entries df's.
filepaths = []
trade_entries_dict = {}

# Uses the 'glob' module to iterate and find all files with .csv, for each file found, the filepath is appended to the filepaths empty list [ ].
for file in glob.glob('*.csv'):
  filepaths.append(file)

# Using a for loop, we iterate through each filepath in 'filepaths' list.
for path in filepaths:
  # Extracts the file's symbol name by splitting and parsing the path at the first index before the period, and saves it to the symbol variable.
  symbol = path.split('.')[0]
  # Reads the file into a DataFrame called 'symbols_df', with the index set to 'datetime'. The parse_dates makes it so the datetime index column is properly a datetime64[ns] object type.
  symbols_df = pd.read_csv(path, index_col='datetime', parse_dates=['datetime'])
  # Calls the 'create_entry_df' function on 'symbols_df' to generate an entirely new DataFrame called 'entry_df', which has the trade entries for a single symbol.
  entry_df = create_entry_df(symbols_df)
  # It adds each DataFrame 'entry_df' which was iterated through for every symbol, and adds it to 'trade_entries_dict', with the symbol name as the key, and the value is the entry trade data.
  trade_entries_dict[symbol] = entry_df

In [6]:
# allows for all rows to be seen if = 'None'. If want less, do = 20.
pd.options.display.max_rows = None

In [ ]:
trade_entries_dict

In [ ]:
price_data_dict

In [ ]:
# merges the price data and trade entries dictionaries. It's shorthand for creating a new dictionary from existing ones.
# for each 'key' in price_data_dict, it makes new key-value pair in the new dict. Rows are aligned based on their common index.
price_and_trades_dict = {key: pd.concat([price_data_dict[key], trade_entries_dict[key]], axis=1) for key in price_data_dict}

In [36]:
price_and_trades_dict

{'A':                   open        high         low       close    volume  \
 datetime                                                               
 2018-04-12   64.132246   65.172855   64.026257   64.748901   2180900   
 2018-04-13   65.037961   65.201758   64.440572   64.758537   2043900   
 2018-04-16   65.355922   65.876227   65.028320   65.683517   2887800   
 2018-04-17   66.097840   66.338721   65.587173   65.943680   2682600   
 2018-04-18   66.598890   67.726213   66.146032   67.013206   2678300   
 2018-04-19   66.945755   67.051736   64.903078   65.490829   2435100   
 2018-04-20   65.558269   65.770246   64.864529   65.086143   1917200   
 2018-04-23   65.240296   66.165280   65.066861   65.587166   2260600   
 2018-04-24   63.968439   65.433001   63.631206   64.209320   3701600   
 2018-04-25   64.257512   64.633286   62.542438   63.399975   4451600   
 2018-04-26   63.804647   64.324951   63.496319   63.949177   1978600   
 2018-04-27   63.920278   64.132255   63.55414

In [39]:
# Function with purpose to create new dictionary which houses the trade results, with an input of the price_and_trades_dict dictionary.
def create_trade_results_dict(price_and_trades_dict):
    trade_results_dict = {}
    # for key, value in dictionary.items
    for symbol, trades_df in price_and_trades_dict.items():
        # Keep only rows with trades (all columns filled in) and creates a copy
        trades_df = trades_df.dropna().copy()

        # Initialize new columns
        trades_df['trade_num'] = range(1, len(trades_df) + 1)
        trades_df['entry_date'] = trades_df.index
        trades_df = trades_df.set_index('trade_num')
        trades_df['exit_date'] = None
        trades_df['days_in_trade'] = None
        trades_df['win_loss'] = None
        trades_df['p_l'] = None

        # Iterate through each row of trades_df using its index and row data
        for idx, row in trades_df.iterrows():
            # Uses '.loc' to access the range of rows starting at 'idx:' and onward.
            # Applies the lambda function to each row, checks when target or stop is hit, and 'idxmax' returns the index of that first row where the lambda function is true, saves that index to the 'exit_row' variable.
            # exit_row = The row index of the first row where the target or stop price is hit.
            # idx = The row index of the current trade entry that we are analyzing in the loop iteration.
            exit_row = trades_df.loc[idx:].apply(lambda x: x['high'] >= row['target'] or x['low'] <= row['stop'], axis=1).idxmax()

            # Calculate exit price, exit date, days in trade, win_loss, and p_l. Format for if statements is ternary as (value if true, condition, else value if false)
            # The format in the brackets, if two terms, is [row, column]
            trades_df.loc[idx, 'exit_price'] = row['target'] if trades_df.loc[exit_row, 'high'] >= row['target'] else row['stop']
            trades_df.loc[idx, 'exit_date'] = trades_df.loc[exit_row, 'entry_date']
            trades_df.loc[idx, 'days_in_trade'] = (trades_df.loc[idx, 'exit_date'] - trades_df.loc[idx, 'entry_date']).days
            trades_df.loc[idx, 'win_loss'] = 1 if trades_df.loc[exit_row, 'high'] >= row['target'] else -1
            trades_df.loc[idx, 'p_l'] = 150 if trades_df.loc[exit_row, 'high'] >= row['target'] else -100

        # Select only columns we want in the final trade results
        selected_columns = ['entry_date', 'entry', 'target', 'stop', 'exit_price', 'exit_date', 'days_in_trade', 'position_size', 'win_loss', 'p_l']
        # Alters trades_df so it only has the selected columns
        trades_df = trades_df[selected_columns]

        # Add the modified DataFrame back to the new dictionary with the original key
        trade_results_dict[symbol] = trades_df

    return trade_results_dict

# Use the function to create the trade_results_dict
trade_results_dict = create_trade_results_dict(price_and_trades_dict)

In [40]:
trade_results_dict

{'A':           entry_date       entry      target        stop  exit_price  \
 trade_num                                                              
 1         2018-05-08   64.922345   66.764057   63.694537   63.694537   
 2         2018-06-15   63.091650   64.760959   61.978778   64.760959   
 3         2018-08-16   63.244033   65.168839   61.960829   65.168839   
 4         2018-11-07   65.807315   68.568931   63.966238   63.966238   
 5         2018-11-20   65.487911   68.233973   63.657203   63.657203   
 6         2019-01-08   66.173811   69.498247   63.957521   69.498247   
 7         2019-06-20   71.112151   72.965377   69.876666   72.965377   
 8         2019-06-24   71.160773   72.966080   69.957235   72.966080   
 9         2019-06-25   70.625999   72.441723   69.415517   72.441723   
 10        2019-06-27   71.228843   73.033108   70.026000   73.033108   
 11        2019-10-18   73.121134   75.451157   71.567786   75.451157   
 12        2019-10-25   73.912126   75.970122 